# Project description

В данном задании вам предстоит осуществить путешевствие в мир Спрингфилда,
где вы сможете познакомиться со всеми любимыми персонажами Симпсонов.

Основным заданием будет обучить классификатор на основе сверточных сетей,
чтобы научиться отличать всех жителей Спрингфилда.
# Dataset description
Обучающая и тестовая выборка состоят из отрывков из мультсериала Симпсоны.
Каждая картинка представлена в формате jpg c необходимой меткой - названием
персонажа изображенного на ней. Тест был поделен на приватную и публичную
часть в соотношении 95/5

В тренировочном датасете примерно по 1000 картинок на каждый класс,
но они отличаются размером.

Метки классов представлены в виде названий папок, в которых лежат картинки.

# Table of content:
1. [__Data preparation__](#data_preparation)
2. [__Create models__](#Create_models)

# <a name='data_preparation'>1. Data preparation</a>

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import functions
import MobileNet_v3
from importlib import reload
functions = reload(functions)
MobileNet_v3 = reload(MobileNet_v3)
from functions import train, accuracy
from MobileNet_v3 import get_model

In [ ]:
import sys
if 'google' in sys.modules:
    from google.colab import drive
    is_in_colab = True
else:
    is_in_colab = False

In [ ]:
if is_in_colab:
    drive.mount('/content/drive')
    !mkdir Data
    !cp drive/My\ Drive/Colab/Stepik/Kaggle/journey-springfield.zip data
    !unzip -q -n data/journey-springfield.zip -d data

In [ ]:
dataset = ImageFolder('Data/train/')

In [ ]:
# look at the image
np.random.seed(42)

fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(8, 8),
                       sharey=True, sharex=True)

for fig_x in ax.flatten():
    random_characters = np.random.choice(len(dataset), 1)[0]
    im, label = dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),
                             dataset.classes[label].split('_')))
    im = im.resize((224, 244))
    fig_x.imshow(im)
    if img_label is not None:
        fig_x.set_title(img_label)
    fig_x.grid(False)

In [ ]:
# Create custom DataLoader
class MyDataLoader:
    def __init__(self, data, indices: list, batch_size: int, transformer=None, shuffle=False):
        assert type(shuffle) is bool, \
            f'shuffle should be bool type, not {type(shuffle)}'
        assert type(batch_size) is int, \
            f'batch_size should be type int, not {type(batch_size)}'

        self.shuffle = shuffle
        self.batch_size = batch_size
        self.indices = indices
        self.data = data
        self.data_len = len(indices)
        self.len_ = int(np.ceil(self.data_len / batch_size))

        self.transformer = transformer
        if transformer is None:
            self.transformer = transforms.ToTensor()

    def __len__(self):
        return self.len_

    def __getitem__(self, index):
        start_index = index * self.batch_size
        end_index = min(self.data_len, start_index + self.batch_size)
        batch_indices = self.indices[start_index: end_index]
        X_batch = []
        y_batch = []
        for batch_index in batch_indices:
            X, y = self.data[batch_index]
            X = self.transformer(X)
            X_batch.append(X)
            y_batch.append(y)
        if len(X_batch) > 1:
            X_batch = torch.stack(X_batch)
        else:
            X_batch = torch.unsqueeze(X_batch[0], 0)
        return X_batch, torch.Tensor(y_batch)

    def __next__(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        for n_batch in range(self.len_):
            return self.__getitem__(n_batch)

In [ ]:
# split data
train_val_indices, test_indices = train_test_split(np.arange(len(dataset)),
                                                   train_size=0.75)

train_indices, val_indices = train_test_split(train_val_indices,
                                              train_size=0.75)


# <a name='Create_models'>2. Create models</a>

## <a name='prepair_praining'>Preparing trainign function</a>

In [ ]:
# light model

class Model(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, dropout, n_classes):
        super().__init__()
        self.snn = nn.Conv2d(in_channel, out_channel, kernel_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(222*222*out_channel, n_classes)
        self.sm = nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        result = self.dropout(self.relu(self.snn(inputs)))
        
        return self.sm(self.linear(self.flatten(result)))

In [ ]:
in_channel = 3
out_channel = 5
kernel_size = 3
dropout = 0.3
output_dim = len(dataset.classes)

model = Model(in_channel, out_channel, kernel_size, dropout, output_dim).to(DEVICE)

In [ ]:
mobilenet = get_model(output_dim, 'small', ).to(DEVICE)

In [ ]:
optimizer = torch.optim.Adam(mobilenet.parameters())

loss_func = nn.NLLLoss().to(DEVICE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.5, patience=20,
    threshold=0.001, cooldown=20, verbose=True)

In [ ]:
IM_SIZE = (224, 224)

train_transformer = transforms.Compose([transforms.Resize(IM_SIZE),
                                        transforms.RandomRotation(15),
                                        transforms.ColorJitter(0.5, 0.5),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])
                                        ])

val_transformer = transforms.Compose([transforms.Resize(IM_SIZE),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])
                                        ])

# Create data loaders
train_loader = MyDataLoader(dataset, train_indices, 64,
                            train_transformer, True)
val_loader = MyDataLoader(dataset, val_indices, 64, val_transformer)
test_loader = MyDataLoader(dataset, test_indices, 64)

In [ ]:
%%time
epoch_count=2
history, best_param = \
        train(mobilenet, train_loader, loss_func, optimizer, epoch_count,
              accuracy, val_loader, scheduler=scheduler)